## 2 Data parsing

First we load `pandas` and `regex` libraries for dataFrame handling and string formatting

In [104]:
import pandas as pd
import re

Then, using `pandas` we load both the csv files

In [2]:
ariba = pd.read_csv('exercise_2/ariba_amr_output.csv')
metadata = pd.read_csv('exercise_2/ncbi_acquired_genes_metadata.csv')

In [7]:
ariba

,name,A7J11_01233+.assembled,A7J11_01233+.ref_seq,A7J11_01233+.pct_id,A7J11_01233+.ctg_cov,aac_3__II-.assembled,aac_3__II-.ref_seq,aac_3__II-.pct_id,aac_3__II-.ctg_cov,aac_6___Ib+.assembled,...,tet_A_.pct_id,tet_A_.ctg_cov,tet_B_.assembled,tet_B_.ref_seq,tet_B_.pct_id,tet_B_.ctg_cov,tet_D_.assembled,tet_D_.ref_seq,tet_D_.pct_id,tet_D_.ctg_cov
0,G18250057,partial,A7J11_01233.NG_047213.1,99.31,22.9,no,NaN,NaN,NaN,yes,...,NaN,NaN,yes,tet_B_.NG_048163.1,100.0,41.5,no,NaN,NaN,NaN
1,G18250058,no,NaN,NaN,NaN,no,NaN,NaN,NaN,no,...,100.0,32.2,no,NaN,NaN,NaN,no,NaN,NaN,NaN
2,G18250059,no,NaN,NaN,NaN,yes,aac_3__IIa.NG_047244.1,99.77,22.6,no,...,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN
3,G18250060,partial,A7J11_01233.NG_047213.1,99.31,24.9,no,NaN,NaN,NaN,yes,...,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN
4,G18250065,partial,A7J11_01233.NG_047213.1,99.31,23.4,no,NaN,NaN,NaN,yes,...,NaN,NaN,yes,tet_B_.NG_048163.1,100.0,28.6,no,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,G18253316,no,NaN,NaN,NaN,no,NaN,NaN,NaN,no,...,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN
78,G18253321,partial,A7J11_01233.NG_047213.1,99.31,9.3,no,NaN,NaN,NaN,yes,...,NaN,NaN,yes,tet_B_.NG_048163.1,100.0,13.9,no,NaN,NaN,NaN
79,G18253323,partial,A7J11_01233.NG_047213.1,99.31,7.8,no,NaN,NaN,NaN,yes,...,NaN,NaN,yes,tet_B_.NG_048163.1,100.0,11.2,no,NaN,NaN,NaN
80,G18253332,no,NaN,NaN,NaN,no,NaN,NaN,NaN,no,...,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN


In [35]:
metadata

,allele,gene_family,refseq_nucleotide_accession,subclass
0,NaN,aac(2')-IIa,NG_047225.1,KASUGAMYCIN
1,NaN,aac(2')-IIb,NG_055672.1,KASUGAMYCIN
2,NaN,aac(2')-Ia,NG_047226.1,GENTAMICIN/TOBRAMCYIN
3,NaN,aac(2')-Ib,NG_047227.1,GENTAMICIN/TOBRAMCYIN
4,NaN,aac(2')-Ic,NG_047229.1,GENTAMICIN/TOBRAMCYIN
...,...,...,...,...
5624,ampC_T-32C,ampC,NZ_CP041538.1,CEPHALOSPORIN
5625,ampC_T-32G,ampC,NZ_CP041538.1,CEPHALOSPORIN
5626,ampC_G-15GG,ampC,NZ_CP041538.1,CEPHALOSPORIN
5627,ampC_T-14TGT,ampC,NZ_CP041538.1,CEPHALOSPORIN


Thereafter, a subset is made from the metadate were only rows with *subclass* CARBAPENEM or CEPHALOSPORIN, and where the *refseq_nucleotide_accession* starts with NG_.
In addition, a list of the Accession IDs is created

In [204]:
refseqs_df = metadata[((metadata['subclass'] == 'CARBAPENEM') | (metadata['subclass'] == 'CEPHALOSPORIN')) & (metadata['refseq_nucleotide_accession'].str.startswith('NG_'))]
refseqs = list(refseqs_df['refseq_nucleotide_accession'])

In [187]:
refseqs_df

,allele,gene_family,refseq_nucleotide_accession,subclass
664,NaN,bla2,NG_047222.1,CARBAPENEM
665,NaN,bla2,NG_047220.1,CARBAPENEM
666,NaN,bla2,NG_047221.1,CARBAPENEM
667,NaN,bla2,NG_047224.1,CARBAPENEM
668,NaN,bla2,NG_055630.1,CARBAPENEM
...,...,...,...,...
4237,NaN,cphA1,NG_047669.1,CARBAPENEM
4238,NaN,cphA1,NG_047670.1,CARBAPENEM
4239,NaN,cphA1,NG_047671.1,CARBAPENEM
4240,NaN,cphA1,NG_047667.1,CARBAPENEM


Since many loci are present in the form of columns, I created a list with the unique names of the loci looping through the column names of the ariba database and removing the extension

In [48]:
index = 1
loci =[]
while index < 200:
    locus = list(ariba.columns)[index].replace('.assembled', '')
    loci.append(locus)
    index += 4

Finally, I created a function that loops over each locus and each row, appending the sample's name, locus, antibiotic for which it is resitant and NCBI Accession ID

In [208]:
def isResistant():
    
    samples_lst, loci_lst, antibiotics_lst, refseqs_lst = [], [], [], []
    
    for row in range(len(ariba)):
        for locus in loci:
                       
            sample = ariba.loc[row]
            name = sample['name']
            assembled = sample[locus + '.assembled']
            ctg = sample[locus + '.ctg_cov']
            
            refseq = str(sample[locus + '.ref_seq'])
            ref_seq_format = re.split("\.", refseq, 1)[-1]
            
            if assembled.startswith('yes') and (ctg >= 10) and (ref_seq_format in refseqs):
                
                antibiotic = refseqs_df[refseqs_df['refseq_nucleotide_accession'] == ref_seq_format]['subclass'].iloc[0]               
                samples_lst.append(name)
                loci_lst.append(locus)
                antibiotics_lst.append(antibiotic)
                refseqs_lst.append(ref_seq_format)
                
    df = pd.DataFrame({'Sample': samples_lst, 'Locus': loci_lst, 'Antibiotic': antibiotics_lst, 'RefSeq_Accession': refseqs_lst})
    return df

Finally, I call the `isResistant` function and save the results in a csv file

In [213]:
ans = isResistant()
ans.head(15)

,Sample,Locus,Antibiotic,RefSeq_Accession
0,G18250057,blaCTX_M_-,CEPHALOSPORIN,NG_048935.1
1,G18250057,blaEC-,CEPHALOSPORIN,NG_049081.1
2,G18250057,blaOXA_-_10,CEPHALOSPORIN,NG_049392.1
3,G18250058,blaCTX_M_-,CEPHALOSPORIN,NG_048935.1
4,G18250058,blaEC-,CEPHALOSPORIN,NG_049085.1
5,G18250059,blaCTX_M_-,CEPHALOSPORIN,NG_048935.1
6,G18250059,blaEC-,CEPHALOSPORIN,NG_049085.1
7,G18250060,blaCTX_M_-,CEPHALOSPORIN,NG_048935.1
8,G18250060,blaEC-,CEPHALOSPORIN,NG_049085.1
9,G18250060,blaOXA_-_10,CEPHALOSPORIN,NG_049392.1


In [218]:
ans.to_csv('resistant_samples.csv', index=False)